In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.1 MB/s eta 0:00:00


In [8]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import missingno

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

seed = 42 # seed 값 설정
random.seed(seed) # 파이썬 난수 생성기
os.environ['PYTHONHASHSEED'] = str(seed) # 해시 시크릿값 고정
np.random.seed(seed) # 넘파이 난수 생성기

torch.manual_seed(seed) # 파이토치 CPU 난수 생성기
torch.backends.cudnn.deterministic = True # 확정적 연산 사용 설정
torch.backends.cudnn.benchmark = False   # 벤치마크 기능 사용 해제
torch.backends.cudnn.enabled = False        # cudnn 기능 사용 해제

if device == 'cuda':
    torch.cuda.manual_seed(seed) # 파이토치 GPU 난수 생성기
    torch.cuda.manual_seed_all(seed) # 파이토치 멀티 GPU 난수 생성기

cpu


In [9]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [10]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

def calculate_metabolic_stability_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    logP = Descriptors.MolLogP(mol)
    # 화합물의 친유성을 측정한 것으로 지질 또는 비극성 환경에서의 용해도를 나타냅니다. 생물학적 막을 통과하는 화합물의 능력을 반영합니다.
    apka = Descriptors.MolWt(mol)
    # 화합물의 산 해리 상수의 추정치로 다양한 pH 조건에서 이온화 거동에 대한 정보를 제공합니다.
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    # 화합물에서 회전 가능한 결합의 수입니다. 이것은 화합물의 유연성과 효소 또는 다른 분자와의 잠재적인 상호 작용에 대한 통찰력을 제공할 수 있습니다.
    num_heteroatoms = Descriptors.NumHeteroatoms(mol)
    # 분자 내 헤테로원자(탄소 및 수소 이외의 원자) 수. 이는 화합물의 반응성과 대사 안정성에 영향을 줄 수 있습니다.
    num_hydrogen_acceptors = Descriptors.NumHAcceptors(mol)
    # 분자 내 수소 결합 수용체의 수. 이들은 결합 및 반응성에 영향을 미치는 다른 분자의 수소 결합 기증자와 상호 작용할 수 있는 사이트입니다.
    num_hydrogen_donors = Descriptors.NumHDonors(mol)
    # 분자 내 수소 결합 기증자의 수입니다. 이들은 수소 결합 상호작용에서 수소 원자를 제공할 수 있는 사이트입니다.
    # morgan_fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    morgan_fingerprint = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    # 분자 하위 구조의 이진 벡터 표현입니다. 이 열에는 화합물과 효소의 상호 작용 및 대사 안정성에 영향을 줄 수 있는 구조적 특징을 포착하는 이진 지문이 포함되어 있습니다.
    morgan_array = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(morgan_fingerprint, morgan_array)

    return logP, apka, num_rotatable_bonds, num_heteroatoms, num_hydrogen_acceptors, num_hydrogen_donors, morgan_array

train[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = train['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

test[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = test['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

train


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,logP,apka,num_rotatable_bonds,num_heteroatoms,num_hydrogen_acceptors,num_hydrogen_donors,morgan_fingerprint
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,3.87744,400.504,8,8,6,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,3.35474,301.415,2,5,4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,1.20450,297.366,3,7,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,3.89356,494.665,5,9,7,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,2.81772,268.316,1,4,3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74,2.74730,396.200,4,11,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37,2.27630,359.389,3,7,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14,2.04130,261.325,5,5,5,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51,1.42720,284.699,4,7,6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [11]:
train['AlogP'].fillna(train['AlogP'].median(), inplace=True)
test['AlogP'].fillna(test['AlogP'].median(), inplace=True)

In [12]:
class CustomDataset(Dataset):
    def __init__(self, data, target_col=None, transform=None, is_test=False):
        self.is_test = is_test
        self.transform = transform
        self.is_test = is_test

        if not self.is_test:
            self.data = self.transform.fit_transform(np.stack(data['morgan_fingerprint']))
        else: # test
            self.data = self.transform.transform(np.stack(data['morgan_fingerprint']))

        if target_col is not None and not self.is_test:
            self.target = data[target_col]

    def __getitem__(self, index):
        features = self.data[index]

        if hasattr(self, 'target'):
            target = self.target[index]
            return torch.tensor(features).to(device).float(), torch.tensor(target).to(device).float().unsqueeze(dim=-1)
        else:
            return torch.tensor(features).to(device).float()

    def __len__(self):
        return len(self.data)


In [13]:
transform = VarianceThreshold(threshold=0.05)

train_MLM = CustomDataset(train, target_col='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(train, target_col='HLM', transform=transform, is_test=False)

In [14]:
X_MLM_train = train_MLM.data
y_MLM_train = train_MLM.target

X_HLM_train = train_HLM.data
y_HLM_train = train_HLM.target

### MLM (XGBoost)

In [64]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),  # 더 작은 학습률로 시도
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 0.6),
        'n_estimators': trial.suggest_int('n_estimators', 400, 1500),  # 범위 확장
        'gamma': trial.suggest_uniform('gamma', 0, 0.2),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 2.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 3.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1, 10),
        'lambda': trial.suggest_uniform('lambda', 0.2, 0.5),
    }

    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=5, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=300)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_MLM_model = XGBRegressor(**best_params)

[I 2023-09-07 18:46:46,146] A new study created in memory with name: no-name-b592ebf7-cb1f-4a09-ae34-2a7bd2080052
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/4214367902.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),  # 더 작은 학습률로 시도
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/4214367902.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/4214367902.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://g

Average RMSE: 15.925017607633055


[I 2023-09-07 18:47:16,188] Trial 1 finished with value: 16.354394848881235 and parameters: {'learning_rate': 0.08661619360732277, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.9728263127342266, 'colsample_bytree': 0.48114526337384855, 'n_estimators': 1064, 'gamma': 0.04387859661308316, 'reg_alpha': 0.2835767206153841, 'reg_lambda': 2.780524361575394, 'scale_pos_weight': 2.3211660902312006, 'lambda': 0.31930929813509795}. Best is trial 0 with value: 15.925017607633055.


Average RMSE: 16.354394848881235


[I 2023-09-07 18:47:21,440] Trial 2 finished with value: 16.051772414080368 and parameters: {'learning_rate': 0.08739581248565376, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.9908576541494534, 'colsample_bytree': 0.17827207961295363, 'n_estimators': 597, 'gamma': 0.16815144380818636, 'reg_alpha': 0.18359918930025199, 'reg_lambda': 2.9292408276177704, 'scale_pos_weight': 7.999587674519194, 'lambda': 0.3369548878429376}. Best is trial 0 with value: 15.925017607633055.


...

Average RMSE: 15.756532036581365


[I 2023-09-07 21:36:01,440] Trial 298 finished with value: 15.775526554752101 and parameters: {'learning_rate': 0.010371152216267412, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.5349139140015703, 'colsample_bytree': 0.5278897366195011, 'n_estimators': 1403, 'gamma': 0.07534760888778055, 'reg_alpha': 1.7436702148085987, 'reg_lambda': 2.1271899759842867, 'scale_pos_weight': 6.60028795261831, 'lambda': 0.3533114414400764}. Best is trial 248 with value: 15.728086380212499.


Average RMSE: 15.775526554752101


[I 2023-09-07 21:36:31,805] Trial 299 finished with value: 15.92316982855308 and parameters: {'learning_rate': 0.02498467670897323, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.5541963383546231, 'colsample_bytree': 0.5548009151407463, 'n_estimators': 1420, 'gamma': 0.0780619405585383, 'reg_alpha': 1.6410825363008776, 'reg_lambda': 2.0418854774121358, 'scale_pos_weight': 6.446316501213089, 'lambda': 0.36478827072939984}. Best is trial 248 with value: 15.728086380212499.


Average RMSE: 15.92316982855308
Best Params: {'learning_rate': 0.0077287406310999306, 'max_depth': 8, 'min_child_weight': 2, 'subsample': 0.567900997184441, 'colsample_bytree': 0.5043000229505461, 'n_estimators': 1430, 'gamma': 0.0656924437984701, 'reg_alpha': 1.268296534134533, 'reg_lambda': 1.6264581356255772, 'scale_pos_weight': 7.437943994430896, 'lambda': 0.33770761497260054}
Best Score: 15.728086380212499


In [ ]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [ ]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [108]:
# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_MLM_train, y_MLM_train, test_size=0.3, random_state=42)

# 교차 검증 수행
scores = cross_val_score(best_MLM_model, X_MLM_train, y_MLM_train, cv=8, scoring=make_scorer(custom_scorer))

# 교차 검증 스코어 출력
print("Cross-Validation RMSE Scores:", scores)
print("Average RMSE:", np.mean(scores))

[0]	validation_0-rmse:52.79793
[1]	validation_0-rmse:52.58368
[2]	validation_0-rmse:52.37159
[3]	validation_0-rmse:52.15136
[4]	validation_0-rmse:51.94930
[5]	validation_0-rmse:51.74677
[6]	validation_0-rmse:51.53630
[7]	validation_0-rmse:51.33265
[8]	validation_0-rmse:51.14440
[9]	validation_0-rmse:50.94794
[10]	validation_0-rmse:50.74381
[11]	validation_0-rmse:50.55148
[12]	validation_0-rmse:50.35178
[13]	validation_0-rmse:50.16308
[14]	validation_0-rmse:49.97496
[15]	validation_0-rmse:49.79386
[16]	validation_0-rmse:49.60529
[17]	validation_0-rmse:49.42745
[18]	validation_0-rmse:49.25076
[19]	validation_0-rmse:49.07205
[20]	validation_0-rmse:48.90683
[21]	validation_0-rmse:48.72968
[22]	validation_0-rmse:48.56285
[23]	validation_0-rmse:48.39142
[24]	validation_0-rmse:48.22168
[25]	validation_0-rmse:48.05668
[26]	validation_0-rmse:47.88762
[27]	validation_0-rmse:47.72558
[28]	validation_0-rmse:47.57772
[29]	validation_0-rmse:47.41657
[30]	validation_0-rmse:47.26035
[31]	validation_0-

/opt/homebrew/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.

/opt/homebrew/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.



[32]	validation_0-rmse:46.94170
[33]	validation_0-rmse:46.78804
[34]	validation_0-rmse:46.62669
[35]	validation_0-rmse:46.47502
[36]	validation_0-rmse:46.32778
[37]	validation_0-rmse:46.18147
[38]	validation_0-rmse:46.03452
[39]	validation_0-rmse:45.89589
[40]	validation_0-rmse:45.74981
[41]	validation_0-rmse:45.60791
[42]	validation_0-rmse:45.46791
[43]	validation_0-rmse:45.32786
[44]	validation_0-rmse:45.18995
[45]	validation_0-rmse:45.05826
[46]	validation_0-rmse:44.92228
[47]	validation_0-rmse:44.78384
[48]	validation_0-rmse:44.64214
[49]	validation_0-rmse:44.51188
[50]	validation_0-rmse:44.38994
[51]	validation_0-rmse:44.26561
[52]	validation_0-rmse:44.12952
[53]	validation_0-rmse:44.00917
[54]	validation_0-rmse:43.88213
[55]	validation_0-rmse:43.75603
[56]	validation_0-rmse:43.63776
[57]	validation_0-rmse:43.51637
[58]	validation_0-rmse:43.39401
[59]	validation_0-rmse:43.28593
[60]	validation_0-rmse:43.16994
[61]	validation_0-rmse:43.06459
[62]	validation_0-rmse:42.94901
[63]	val

[283]	validation_0-rmse:33.56844
[284]	validation_0-rmse:33.55591
[285]	validation_0-rmse:33.54342
[286]	validation_0-rmse:33.52911
[287]	validation_0-rmse:33.51519
[288]	validation_0-rmse:33.50984
[289]	validation_0-rmse:33.49837
[290]	validation_0-rmse:33.49243
[291]	validation_0-rmse:33.48096
[292]	validation_0-rmse:33.47205
[293]	validation_0-rmse:33.46347
[294]	validation_0-rmse:33.45549
[295]	validation_0-rmse:33.44627
[296]	validation_0-rmse:33.43855
[297]	validation_0-rmse:33.42868
[298]	validation_0-rmse:33.41980
[299]	validation_0-rmse:33.40618
[300]	validation_0-rmse:33.39865
[301]	validation_0-rmse:33.39322
[302]	validation_0-rmse:33.38514
[303]	validation_0-rmse:33.37285
[304]	validation_0-rmse:33.36675
[305]	validation_0-rmse:33.36114
[306]	validation_0-rmse:33.35457
[307]	validation_0-rmse:33.34773
[308]	validation_0-rmse:33.33682
[309]	validation_0-rmse:33.32863
[310]	validation_0-rmse:33.32124
[311]	validation_0-rmse:33.31199
[312]	validation_0-rmse:33.30957
[313]	vali

[532]	validation_0-rmse:32.53571
[533]	validation_0-rmse:32.53525
[534]	validation_0-rmse:32.53468
[535]	validation_0-rmse:32.53546
[536]	validation_0-rmse:32.52996
[537]	validation_0-rmse:32.53198
[538]	validation_0-rmse:32.53328
[539]	validation_0-rmse:32.53402
[540]	validation_0-rmse:32.53180
[541]	validation_0-rmse:32.53093
[542]	validation_0-rmse:32.52826
[543]	validation_0-rmse:32.52796
[544]	validation_0-rmse:32.52633
[545]	validation_0-rmse:32.52639
[546]	validation_0-rmse:32.52499
[547]	validation_0-rmse:32.52180
[548]	validation_0-rmse:32.52055
[549]	validation_0-rmse:32.51848
[550]	validation_0-rmse:32.51719
[551]	validation_0-rmse:32.51596
[552]	validation_0-rmse:32.51459
[553]	validation_0-rmse:32.51345
[554]	validation_0-rmse:32.51253
[555]	validation_0-rmse:32.50922
[556]	validation_0-rmse:32.50755
[557]	validation_0-rmse:32.50571
[558]	validation_0-rmse:32.50359
[559]	validation_0-rmse:32.50208
[560]	validation_0-rmse:32.50305
[561]	validation_0-rmse:32.50308
[562]	vali

[1028]	validation_0-rmse:32.36330
[1029]	validation_0-rmse:32.36434
Validation RMSE: 32.34268273323046


### HLM (XGBoost)

In [70]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),  # 더 작은 학습률로 시도
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 0.6),
        'n_estimators': trial.suggest_int('n_estimators', 400, 1500),  # 범위 확장
        'gamma': trial.suggest_uniform('gamma', 0, 0.2),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 2.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 3.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1, 10),
        'lambda': trial.suggest_uniform('lambda', 0.2, 0.5),
    }


    # 데이터 분할
    X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.2, random_state=42)

    
    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=5, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=300)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_HLM_model = XGBRegressor(**best_params)

[I 2023-09-07 21:45:59,056] A new study created in memory with name: no-name-129af82b-b35d-453b-be30-77cb061a3b5d
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/684393978.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),  # 더 작은 학습률로 시도
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/684393978.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/684393978.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://gith

Average RMSE: 16.52394909366682


[I 2023-09-07 21:46:35,778] Trial 1 finished with value: 16.05899736634124 and parameters: {'learning_rate': 0.04097190826464154, 'max_depth': 8, 'min_child_weight': 8, 'subsample': 0.5822617918564024, 'colsample_bytree': 0.23500905342649983, 'n_estimators': 1033, 'gamma': 0.11692893566350367, 'reg_alpha': 0.05331492652684933, 'reg_lambda': 2.857314011325901, 'scale_pos_weight': 2.63567100398409, 'lambda': 0.2646310598444438}. Best is trial 1 with value: 16.05899736634124.


Average RMSE: 16.05899736634124


[I 2023-09-07 21:46:52,850] Trial 2 finished with value: 15.952808216292581 and parameters: {'learning_rate': 0.02716467160007302, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.9874659079421009, 'colsample_bytree': 0.3077974509772964, 'n_estimators': 1385, 'gamma': 0.12328154296105709, 'reg_alpha': 0.6479994035913685, 'reg_lambda': 2.916473671015526, 'scale_pos_weight': 7.8052127820522745, 'lambda': 0.3074582732809965}. Best is trial 2 with value: 15.952808216292581.


...

Average RMSE: 15.754950108070735


[I 2023-09-08 00:53:19,982] Trial 298 finished with value: 15.828551588895214 and parameters: {'learning_rate': 0.0040429508971851855, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.5638793351914438, 'colsample_bytree': 0.25689592254437005, 'n_estimators': 1349, 'gamma': 0.19311066392585263, 'reg_alpha': 0.4872660584652521, 'reg_lambda': 2.6916627579267867, 'scale_pos_weight': 6.9742655532591415, 'lambda': 0.37526941392154356}. Best is trial 192 with value: 15.71471105574769.


Average RMSE: 15.828551588895214


[I 2023-09-08 00:53:55,108] Trial 299 finished with value: 15.861959941616092 and parameters: {'learning_rate': 0.01144807960795323, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.8199729669860331, 'colsample_bytree': 0.4196784014789499, 'n_estimators': 1404, 'gamma': 0.19881805043456666, 'reg_alpha': 1.3099900930094235, 'reg_lambda': 2.5466800782362915, 'scale_pos_weight': 5.035166596265466, 'lambda': 0.4673531977794356}. Best is trial 192 with value: 15.71471105574769.


Average RMSE: 15.861959941616092
Best Params: {'learning_rate': 0.005152914655533061, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.5261326050478413, 'colsample_bytree': 0.4849248831164231, 'n_estimators': 1437, 'gamma': 0.13945536623177748, 'reg_alpha': 1.8002069028133991, 'reg_lambda': 1.3850495493638941, 'scale_pos_weight': 6.525683795195247, 'lambda': 0.38982571712726277}
Best Score: 15.71471105574769


In [75]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [76]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [107]:
# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.3, random_state=42)

# 교차 검증 수행
scores = cross_val_score(best_HLM_model, X_HLM_train, y_HLM_train, cv=8, scoring=make_scorer(custom_scorer))

# 교차 검증 스코어 출력
print("Cross-Validation RMSE Scores:", scores)
print("Average RMSE:", np.mean(scores))

[0]	validation_0-rmse:64.33365
[1]	validation_0-rmse:64.11020
[2]	validation_0-rmse:63.89068
[3]	validation_0-rmse:63.66329
[4]	validation_0-rmse:63.43937
[5]	validation_0-rmse:63.21880
[6]	validation_0-rmse:62.99887
[7]	validation_0-rmse:62.78017
[8]	validation_0-rmse:62.56230
[9]	validation_0-rmse:62.34860
[10]	validation_0-rmse:62.12712
[11]	validation_0-rmse:61.91978
[12]	validation_0-rmse:61.70190
[13]	validation_0-rmse:61.49283
[14]	validation_0-rmse:61.29053
[15]	validation_0-rmse:61.07927
[16]	validation_0-rmse:60.86918
[17]	validation_0-rmse:60.67054
[18]	validation_0-rmse:60.47170
[19]	validation_0-rmse:60.27317
[20]	validation_0-rmse:60.07528
[21]	validation_0-rmse:59.87931
[22]	validation_0-rmse:59.68611
[23]	validation_0-rmse:59.48981
[24]	validation_0-rmse:59.30616
[25]	validation_0-rmse:59.11739
[26]	validation_0-rmse:58.92012
[27]	validation_0-rmse:58.72231
[28]	validation_0-rmse:58.53497
[29]	validation_0-rmse:58.34413


/opt/homebrew/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.

/opt/homebrew/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.



[30]	validation_0-rmse:58.15540
[31]	validation_0-rmse:57.96531
[32]	validation_0-rmse:57.77973
[33]	validation_0-rmse:57.59639
[34]	validation_0-rmse:57.41569
[35]	validation_0-rmse:57.23809
[36]	validation_0-rmse:57.06154
[37]	validation_0-rmse:56.88510
[38]	validation_0-rmse:56.70502
[39]	validation_0-rmse:56.53343
[40]	validation_0-rmse:56.36071
[41]	validation_0-rmse:56.19228
[42]	validation_0-rmse:56.02075
[43]	validation_0-rmse:55.85213
[44]	validation_0-rmse:55.68909
[45]	validation_0-rmse:55.52081
[46]	validation_0-rmse:55.35195
[47]	validation_0-rmse:55.18288
[48]	validation_0-rmse:55.01733
[49]	validation_0-rmse:54.85617
[50]	validation_0-rmse:54.69193
[51]	validation_0-rmse:54.53810
[52]	validation_0-rmse:54.37247
[53]	validation_0-rmse:54.21420
[54]	validation_0-rmse:54.05906
[55]	validation_0-rmse:53.90055
[56]	validation_0-rmse:53.74721
[57]	validation_0-rmse:53.59437
[58]	validation_0-rmse:53.44054
[59]	validation_0-rmse:53.28837
[60]	validation_0-rmse:53.13318
[61]	val

[281]	validation_0-rmse:36.50684
[282]	validation_0-rmse:36.48087
[283]	validation_0-rmse:36.45729
[284]	validation_0-rmse:36.43135
[285]	validation_0-rmse:36.40155
[286]	validation_0-rmse:36.37316
[287]	validation_0-rmse:36.34981
[288]	validation_0-rmse:36.32450
[289]	validation_0-rmse:36.29735
[290]	validation_0-rmse:36.27301
[291]	validation_0-rmse:36.24884
[292]	validation_0-rmse:36.22333
[293]	validation_0-rmse:36.20389
[294]	validation_0-rmse:36.17915
[295]	validation_0-rmse:36.15277
[296]	validation_0-rmse:36.12120
[297]	validation_0-rmse:36.09295
[298]	validation_0-rmse:36.06943
[299]	validation_0-rmse:36.04216
[300]	validation_0-rmse:36.02101
[301]	validation_0-rmse:35.99837
[302]	validation_0-rmse:35.97248
[303]	validation_0-rmse:35.95381
[304]	validation_0-rmse:35.92763
[305]	validation_0-rmse:35.90593
[306]	validation_0-rmse:35.88343
[307]	validation_0-rmse:35.86246
[308]	validation_0-rmse:35.83753
[309]	validation_0-rmse:35.81440
[310]	validation_0-rmse:35.79688
[311]	vali

[530]	validation_0-rmse:33.43713
[531]	validation_0-rmse:33.43153
[532]	validation_0-rmse:33.42803
[533]	validation_0-rmse:33.42131
[534]	validation_0-rmse:33.41823
[535]	validation_0-rmse:33.41610
[536]	validation_0-rmse:33.41106
[537]	validation_0-rmse:33.40689
[538]	validation_0-rmse:33.40510
[539]	validation_0-rmse:33.40192
[540]	validation_0-rmse:33.39573
[541]	validation_0-rmse:33.39213
[542]	validation_0-rmse:33.39191
[543]	validation_0-rmse:33.38657
[544]	validation_0-rmse:33.38034
[545]	validation_0-rmse:33.37907
[546]	validation_0-rmse:33.37303
[547]	validation_0-rmse:33.37018
[548]	validation_0-rmse:33.36501
[549]	validation_0-rmse:33.35823
[550]	validation_0-rmse:33.35491
[551]	validation_0-rmse:33.35297
[552]	validation_0-rmse:33.34835
[553]	validation_0-rmse:33.34280
[554]	validation_0-rmse:33.34069
[555]	validation_0-rmse:33.33304
[556]	validation_0-rmse:33.32765
[557]	validation_0-rmse:33.32277
[558]	validation_0-rmse:33.32005
[559]	validation_0-rmse:33.31691
[560]	vali

[779]	validation_0-rmse:32.91628
[780]	validation_0-rmse:32.91713
[781]	validation_0-rmse:32.91742
[782]	validation_0-rmse:32.91532
[783]	validation_0-rmse:32.91552
[784]	validation_0-rmse:32.91478
[785]	validation_0-rmse:32.91191
[786]	validation_0-rmse:32.91321
[787]	validation_0-rmse:32.91152
[788]	validation_0-rmse:32.90925
[789]	validation_0-rmse:32.90675
[790]	validation_0-rmse:32.90690
[791]	validation_0-rmse:32.90585
[792]	validation_0-rmse:32.90424
[793]	validation_0-rmse:32.90175
[794]	validation_0-rmse:32.90231
[795]	validation_0-rmse:32.90099
[796]	validation_0-rmse:32.90158
[797]	validation_0-rmse:32.90111
[798]	validation_0-rmse:32.90075
[799]	validation_0-rmse:32.89904
[800]	validation_0-rmse:32.89817
[801]	validation_0-rmse:32.89873
[802]	validation_0-rmse:32.89714
[803]	validation_0-rmse:32.89762
[804]	validation_0-rmse:32.89561
[805]	validation_0-rmse:32.89647
[806]	validation_0-rmse:32.89745
[807]	validation_0-rmse:32.89702
[808]	validation_0-rmse:32.89480
[809]	vali

[1027]	validation_0-rmse:32.74913
[1028]	validation_0-rmse:32.74877
[1029]	validation_0-rmse:32.74900
[1030]	validation_0-rmse:32.74870
[1031]	validation_0-rmse:32.74767
[1032]	validation_0-rmse:32.74594
[1033]	validation_0-rmse:32.74502
[1034]	validation_0-rmse:32.74167
[1035]	validation_0-rmse:32.74233
[1036]	validation_0-rmse:32.74333
[1037]	validation_0-rmse:32.74192
[1038]	validation_0-rmse:32.74293
[1039]	validation_0-rmse:32.74124
[1040]	validation_0-rmse:32.73933
[1041]	validation_0-rmse:32.73894
[1042]	validation_0-rmse:32.73707
[1043]	validation_0-rmse:32.73640
[1044]	validation_0-rmse:32.73761
[1045]	validation_0-rmse:32.73598
[1046]	validation_0-rmse:32.73598
[1047]	validation_0-rmse:32.73685
[1048]	validation_0-rmse:32.73494
[1049]	validation_0-rmse:32.73429
[1050]	validation_0-rmse:32.73567
[1051]	validation_0-rmse:32.73439
[1052]	validation_0-rmse:32.73371
[1053]	validation_0-rmse:32.73407
[1054]	validation_0-rmse:32.73577
[1055]	validation_0-rmse:32.73767
[1056]	validat

[1268]	validation_0-rmse:32.69437
[1269]	validation_0-rmse:32.69491
[1270]	validation_0-rmse:32.69418
[1271]	validation_0-rmse:32.69518
[1272]	validation_0-rmse:32.69663
[1273]	validation_0-rmse:32.69437
[1274]	validation_0-rmse:32.69596
[1275]	validation_0-rmse:32.69647
[1276]	validation_0-rmse:32.69617
[1277]	validation_0-rmse:32.69647
[1278]	validation_0-rmse:32.69509
[1279]	validation_0-rmse:32.69608
[1280]	validation_0-rmse:32.69611
[1281]	validation_0-rmse:32.69464
[1282]	validation_0-rmse:32.69589
[1283]	validation_0-rmse:32.69502
[1284]	validation_0-rmse:32.69425
[1285]	validation_0-rmse:32.69682
[1286]	validation_0-rmse:32.69588
[1287]	validation_0-rmse:32.69590
[1288]	validation_0-rmse:32.69604
[1289]	validation_0-rmse:32.69703
[1290]	validation_0-rmse:32.69624
[1291]	validation_0-rmse:32.69544
[1292]	validation_0-rmse:32.69439
[1293]	validation_0-rmse:32.69460
[1294]	validation_0-rmse:32.69620
[1295]	validation_0-rmse:32.69705
[1296]	validation_0-rmse:32.69805
[1297]	validat

### MLM (RandomForest)

In [94]:
from sklearn.ensemble import RandomForestRegressor

# RMSE를 사용하는 스코어 함수 정의 (이전과 동일)
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의 (랜덤 포레스트용으로 수정 및 추가된 하이퍼파라미터 포함)
def objective(trial):
    param_space = {
        'n_jobs': -1,  # 병렬 처리를 위해 모든 CPU 코어 사용
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_uniform('min_samples_split', 0.1, 1.0),
        'min_samples_leaf': trial.suggest_uniform('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 100),
    }

    model = RandomForestRegressor(**param_space)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=5, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=500)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 랜덤 포레스트 모델 생성
best_MLM_Rmodel = RandomForestRegressor(**best_params)

[I 2023-09-08 01:20:58,858] A new study created in memory with name: no-name-f180f306-e828-4852-a1f3-df9d87d734db
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 01:21:01,116] Trial 0 finished with value: 17.5854230903

Average RMSE: 17.585423090327318


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 01:21:03,028] Trial 1 finished with value: 17.415464223962214 and parameters: {'n_estimators': 457, 'max_depth': 6, 'min_samples_split': 0.6480952676154511, 'min_samples_

Average RMSE: 17.415464223962214


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 01:21:05,380] Trial 2 finished with value: 17.62339489128102 and parameters: {'n_estimators': 323, 'max_depth': 5, 'min_samples_split': 0.6276079506277227, 'min_samples_l

...

Average RMSE: 17.120161937173826


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 02:02:34,825] Trial 498 finished with value: 17.605599599498866 and parameters: {'n_estimators': 900, 'max_depth': 7, 'min_samples_split': 0.5704130823255003, 'min_sample

Average RMSE: 17.605599599498866


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2936665158.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 02:02:42,036] Trial 499 finished with value: 17.09636169861539 and parameters: {'n_estimators': 858, 'max_depth': 7, 'min_samples_split': 0.12166269070208771, 'min_sample

Average RMSE: 17.09636169861539
Best Params: {'n_estimators': 879, 'max_depth': 7, 'min_samples_split': 0.11471011651865876, 'min_samples_leaf': 0.10026969268802727, 'max_features': 0.5376160379232956, 'bootstrap': False, 'max_leaf_nodes': 46}
Best Score: 17.08228160626309


In [95]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [96]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [106]:
from sklearn.metrics import mean_squared_error

# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_MLM_train, y_MLM_train, test_size=0.3, random_state=42)

# 모델 훈련
best_MLM_Rmodel.fit(X_train, y_train)

# 예측
y_pred = best_MLM_Rmodel.predict(X_valid)

# 평가 지표 (RMSE) 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Validation RMSE:", rmse)

# 중요한 하이퍼파라미터 출력
print("Best Hyperparameters:")
print("n_estimators:", best_MLM_Rmodel.n_estimators)
print("max_depth:", best_MLM_Rmodel.max_depth)
# 다른 중요한 하이퍼파라미터들도 필요에 따라 출력할 수 있습니다.

Validation RMSE: 34.879641281728375
Best Hyperparameters:
n_estimators: 879
max_depth: 7


### HLM (RandomForest)

In [109]:
from sklearn.ensemble import RandomForestRegressor

# RMSE를 사용하는 스코어 함수 정의 (이전과 동일)
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의 (랜덤 포레스트용으로 수정 및 추가된 하이퍼파라미터 포함)
def objective(trial):
    param_space = {
        'n_jobs': -1,  # 병렬 처리를 위해 모든 CPU 코어 사용
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_uniform('min_samples_split', 0.1, 1.0),
        'min_samples_leaf': trial.suggest_uniform('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 100),
    }

    model = RandomForestRegressor(**param_space)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_HLM_train, y_HLM_train, cv=5, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse


# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=500)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 랜덤 포레스트 모델 생성
best_HLM_Rmodel = RandomForestRegressor(**best_params)

[I 2023-09-08 03:30:47,194] A new study created in memory with name: no-name-7f63c465-ff0b-4305-b825-91df444cd2bf
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 03:30:49,357] Trial 0 finished with value: 17.7594324087

Average RMSE: 17.759432408773193


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 03:30:51,005] Trial 1 finished with value: 18.038647432426952 and parameters: {'n_estimators': 771, 'max_depth': 8, 'min_samples_split': 0.7608933179942674, 'min_samples_

Average RMSE: 18.038647432426952


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 03:30:53,172] Trial 2 finished with value: 17.694930270898375 and parameters: {'n_estimators': 449, 'max_depth': 6, 'min_samples_split': 0.4189605116382341, 'min_samples_

...

Average RMSE: 17.45719418309026


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 03:53:06,331] Trial 498 finished with value: 17.589619197008552 and parameters: {'n_estimators': 710, 'max_depth': 8, 'min_samples_split': 0.4976745690737865, 'min_sample

Average RMSE: 17.589619197008552


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_22386/2553745308.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-08 03:53:08,760] Trial 499 finished with value: 17.467604472015466 and parameters: {'n_estimators': 391, 'max_depth': 8, 'min_samples_split': 0.20259829969352744, 'min_sampl

Average RMSE: 17.467604472015466
Best Params: {'n_estimators': 400, 'max_depth': 9, 'min_samples_split': 0.21298974207456567, 'min_samples_leaf': 0.10015560692651472, 'max_features': 0.4352708466287485, 'bootstrap': False, 'max_leaf_nodes': 16}
Best Score: 17.41320194938982


In [110]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [111]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [112]:
# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.3, random_state=42)

# 모델 훈련
best_HLM_Rmodel.fit(X_train, y_train)

# 예측
y_pred = best_HLM_Rmodel.predict(X_valid)

# 평가 지표 (RMSE) 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Validation RMSE:", rmse)

# 중요한 하이퍼파라미터 출력
print("Best Hyperparameters:")
print("n_estimators:", best_HLM_Rmodel.n_estimators)
print("max_depth:", best_HLM_Rmodel.max_depth)
# 다른 중요한 하이퍼파라미터들도 필요에 따라 출력할 수 있습니다.

Validation RMSE: 34.97555295106262
Best Hyperparameters:
n_estimators: 400
max_depth: 9


### 예측값 구하기

In [113]:
test_MLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)
test_HLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)

X_MLM_test = test_MLM.data
X_HLM_test = test_HLM.data

# 최적의 에폭에서 중단된 모델을 사용하여 예측 수행
y_MLM_pred = best_MLM_model.predict(X_MLM_test)
y_HLM_pred = best_HLM_model.predict(X_HLM_test)

y_MLM_Rpred = best_MLM_Rmodel.predict(X_MLM_test)
y_HLM_Rpred = best_HLM_Rmodel.predict(X_HLM_test)

In [116]:
submission = pd.read_csv('./data/sample_submission.csv')

submission['MLM'] = y_MLM_Rpred
submission['HLM'] = y_HLM_Rpred
submission

,id,MLM,HLM
0,TEST_000,42.465305,53.064616
1,TEST_001,33.601048,53.571257
2,TEST_002,41.643945,47.861793
3,TEST_003,30.028887,51.385959
4,TEST_004,35.270109,58.182661
...,...,...,...
478,TEST_478,39.970450,63.392330
479,TEST_479,37.292139,61.269696
480,TEST_480,24.569577,42.344701
481,TEST_481,46.100019,57.838089


In [117]:
submission.to_csv('./submissions/Optuna+RandomForest_submission_1.csv', index=False)